In [ ]:
from pathlib import Path
import numpy as np
import soundfile
import matplotlib.pyplot as plt
import librosa
from pprint import pprint
import scipy.ndimage
import scipy.signal
import logging
import itertools
import scipy.sparse
import datetime
import os
import pickle
from tqdm import tqdm
import time
from tabulate import tabulate

In [ ]:
RESULTS_DIR = Path("/data5/anasynth_nonbp/andre/reverse-dj-mix/results")
UNMIXDB_PATH = Path("/data2/anasynth_nonbp/schwarz/abc-dj/data/unmixdb-zenodo")

In [ ]:
results = []
for i in tqdm(RESULTS_DIR.glob("*/*/results.pickle")):
    with open(i, "rb") as f:
        results.append(pickle.load(f))

In [ ]:
results[8]

In [ ]:
import param_estimator


def dump_result(result: dict):
    for i in range(3):
        for g_estor in result["highparams"][i]:
            if g_estor == "real":
                continue
            for w_estor in result["highparams"][i][g_estor]:
                print(i, g_estor, w_estor)
                print(f"{"param":<20} {"real":>8} {"est":>8} {"err":>8}")
                for p in result["highparams"][i][g_estor][w_estor]:
                    real = result["highparams"][i]["real"][p]
                    est = result["highparams"][i][g_estor][w_estor][p]
                    err = param_estimator.error(est, real)
                    print(f"{p:<20} {real: 8.2f} {est: 8.2f} {err: 8.2e}")
dump_result(results[0])

In [ ]:
import seaborn as sns
import pandas as pd

def aggregate(p, g_estor, w_estor, results):
    ret = []
    for r in results:
        for i in range(3):
            try:
                real = r["highparams"][i]["real"][p]
                est = r["highparams"][i][g_estor][w_estor][p]
                err = param_estimator.error(est, real)
                ret.append(err)
            except KeyError:
                pass
    return ret

df = pd.DataFrame(columns=['estors', 'param', 'error'])
for param in ["track_start","fadein_start","fadein_stop","fadeout_start","fadeout_stop"]:
    for g_estor in ["GainEstimator.SUM", "GainEstimator.MAX", "GainEstimator.RELSUM"]:
        for w_estor in ["WarpEstimator.ARGMAX", "WarpEstimator.CENTER_OF_MASS"]:
            errors = aggregate(param, g_estor, w_estor, results)
            new_rows = pd.DataFrame({'estors': g_estor+" "+w_estor, "param": param, 'error': errors})
            df = pd.concat([df, new_rows], ignore_index=True)
sns.boxplot(x='param', y='error', hue='estors', data=df, log_scale=True)
plt.title(f'High param estimation errors')
plt.ylim(1e-12,1e2)
plt.show()